Pipeline:
- Filter issues that interest us (healthcare, racism + immigration, housing, guns + criminal justice).
- For each of the issues (each issue a corpus), vectorize/apply tf-idf to actors and claims.
- Create a criteria for keeping the top n-grams.
- Add metadata to resulting DTMs (the relevant thing is the year and the month).